In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
import joblib

# Load scalers
scaler_X = joblib.load('scaler_X.pkl')
scaler_y = joblib.load('scaler_y.pkl')

SEQUENCE_LENGTH = 600
PLOT_SAVE_DIR = 'predicted_vs_actual_plots'

# Load the trained model
def load_trained_model(model_path):
    return load_model(model_path)

# Process the file and prepare segments
def process_file(file_path):
    df = pd.read_csv(file_path)
    df['Time'] = df['Time'] - df['Time'].iloc[0]
    df['Trip fuel consumption'] = df['Trip fuel consumption'] - df['Trip fuel consumption'].iloc[0]
    df['Acceleration'] = df['Speed'].diff().fillna(0)
    features = df[['Engine speed', 'Speed', 'slope', 'Acceleration']]
    df['Momentary fuel consumption'] = df['Trip fuel consumption'].diff().fillna(0)
    target = df['Momentary fuel consumption']
    return features, target

# Pad and normalize the data
def pad_and_normalize(data, scaler, sequence_length=SEQUENCE_LENGTH):
    padded_data = pad_sequences(data, maxlen=sequence_length, dtype='float32', padding='post', truncating='post')
    # print(padded_data)
    min_val_x = [0,0,-10,-150]
    max_val_x = [8000,150,10,150]
    for i in range(padded_data.shape[-1]):
      padded_data[:,:,i] = (padded_data[:,:,i] - min_val_x[i])/(max_val_x[i]-min_val_x[i])
    # normalized_data = scaler.transform(padded_data.reshape(-1, padded_data.shape[-1])).reshape(padded_data.shape)
    # print('\n\n\n',padded_data)
    normalized_data = padded_data
    return normalized_data

# Predict and plot the results
def plot_predicted_vs_real(input_file, model, scaler_X, scaler_y):
    features, actual_values = process_file(input_file)
    num_segments = len(features) // SEQUENCE_LENGTH
    predictions = []

    for i in range(num_segments):
        segment = features.iloc[i * SEQUENCE_LENGTH:(i + 1) * SEQUENCE_LENGTH]
        segment_normalized = pad_and_normalize([segment.values], scaler_X)
        segment_predictions = model.predict(segment_normalized)
        # predictions.extend(scaler_y.inverse_transform(segment_predictions.reshape(-1, 1)))
        predictions.extend((segment_predictions*22000 - 2000).reshape(-1, 1))
        # print(predictions)


    # Handle any remaining data
    remainder = len(features) % SEQUENCE_LENGTH
    if remainder != 0:
        last_segment = features.iloc[-remainder:]
        last_segment_normalized = pad_and_normalize([last_segment.values], scaler_X)
        last_segment_predictions = model.predict(last_segment_normalized)
        # predictions.extend(scaler_y.inverse_transform(last_segment_predictions.reshape(-1, 1)))
        predictions.extend((segment_predictions*22000 - 2000).reshape(-1, 1))

    plt.figure(figsize=(10, 6))
    plt.plot(np.cumsum(actual_values.values, axis=0), label='Real', color='blue')
    plt.plot(np.cumsum(predictions[:len(actual_values)], axis=0), label='Predicted', color='red')
    plt.xlabel('Index')
    plt.ylabel('Fuel Consumption')
    plt.title('Predicted vs Real Fuel Consumption')
    plt.legend()

    directory, filename = os.path.split(input_file)
    plot_filename = os.path.join(directory, f'{os.path.splitext(filename)[0]}_predicted_vs_real.png')
    plt.savefig(plot_filename)
    plt.close()

    print(f"Plot saved as: {plot_filename}")
    predictions = predictions[:len(actual_values)]

    # Save predictions and actual values to CSV
    results_df = pd.DataFrame({
        'Speed': features["Speed"],
        'Actual': np.cumsum(actual_values.values, axis=0),
        'Predicted': np.cumsum(predictions[:len(actual_values)], axis=0).flatten()  # flatten the cumulative predictions
    })

    directory, filename = os.path.split(input_file)
    csv_filename = os.path.join(directory, f'{os.path.splitext(filename)[0]}_predicted_vs_real.csv')
    results_df.to_csv(csv_filename, index=False)

# Paths to model and input file
model_path = '/content/modelBLSTM_new_majid_added.h5'
input_file_path = '/content/NEDC_1000_slope_added.csv'

# Load model and predict
model = load_trained_model(model_path)
plot_predicted_vs_real(input_file_path, model, scaler_X, scaler_y)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 676ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
Plot saved as: /content/NEDC_1000_slope_added_predicted_vs_real.png


In [ ]:
from google.colab import drive
drive.mount('/content/drive')